**"" Program Name : Sig De&Ext(No graph, Fixed Interval)**  
**"" Programmer : Joo Icksoo**  
**"" Version No. : 1.0**  
**"" Revision Date : 21.03.17**  
**"" Function :**

    본 프로그램은 사용자의 동작을 감지하여 저장하는 프로그램이며 동작 횟수를 입력받아 N번의 동작을 한번에 저장 한다.
    사용자에게 동작을 하도록 지시하며 한번의 동작이 끝나고 입력받은 동작 간격이 지난 후에 다음 동작을 요청한다.
    입력은 센서 A,B로부터의 신호이고  A,B 센서의 출력이 입력으로 들어온다.
    저장된 파일은 '../../../Data/Original Data/'밑의 디렉토리에 시간 디렉토리가 생성되어 A,B센서의 데이터가 각각 저장된다.

In [ ]:

import os
import sys
import time
from datetime import datetime

import matplotlib.pyplot as plt
import nidaqmx
import numpy as np
from IPython import display
from IPython.display import clear_output
from nidaqmx import constants
from pandas import DataFrame as df
from playsound import playsound
from scipy import signal


def Graph_drawing(Frame_start, Frame_end, A_threshold_up, A_threshold_down, B_threshold_up, B_threshold_down):
    '''
    들어온 센서의 출력에서 동작 부분을 감지하고 감지한 부분을 그래프로 나타내는 함수이다.
    '''
    clear_output(wait=True)
    global check
    global A_value
    global B_value
    global start_point
    global end_point
    global Motion_occur

    A_temp = df(A_value[Frame_start - 1000:Frame_end + 1000])
    B_temp = df(B_value[Frame_start - 1000:Frame_end + 1000])

    fig = plt.figure(figsize=(9, 3))
    ax1 = fig.add_subplot(1, 2, 1)
    ax2 = fig.add_subplot(1, 2, 2)
    ax1.plot(A_temp, label='A', c='r')
    ax1.plot(B_temp, label='B', c='b')
    ax1.axvline(x=1000)
    ax1.axvline(x=len(A_temp) - 1000)
    ax1.axhline(y=A_threshold_up, c='r')
    ax1.axhline(y=A_threshold_down, c='r')
    ax1.axhline(y=B_threshold_up, c='b')
    ax1.axhline(y=B_threshold_down, c='b')
    ax1.legend(loc='upper right')
    #ax1.set_ylim(-0.2,0.2)
    ax2.plot(np.array(A_temp) - np.array(B_temp), c='black')
    ax2.axvline(x=1000)
    ax2.axvline(x=len(A_temp) - 1000)
    display.display(fig)
    #fig.savefig(path + 'Graphic/' + str(count) + '.png', dpi=fig.dpi)
    #plt.clf()

In [ ]:
mod = sys.modules[__name__]
%matplotlib inline

Initial_threshold = float(input("초기 문턱치를 입력하세요 : "))
human = input("실험자의 이름을 입력하세요 : ")
motion = input("실험할 동작 종류를 입력하세요 : ")
limit = int(input("실험할 동작 횟수를 입력하세요 : "))
interval_period = float(input("동작이 발생한 후 다음 동작까지의 간격을 초 단위로 입력하세요 : "))
flag_1 = int(input("그래프를 그릴 경우 1을 아니면 0을 입력하세요"))

A_value = []
B_value = []

Motion_occur = 0

Frame_start = 0
Frame_end = 0
#Variables that represent the starting point of the frame

data = [[], []]

sfreq = 1000

start_point = []
end_point = []

inc = 0  #중요

interval = 5000
count = 0
check = 0

start = time.time()
print("동작 실험을 시작합니다. 대기해 주세요")
playsound("start.mp3")

with nidaqmx.Task() as task:
    task.__init__(new_task_name='EPS Signal')
    ch_A = task.ai_channels.add_ai_voltage_chan("Dev1/ai5", max_val=10, min_val=-10)
    ch_B = task.ai_channels.add_ai_voltage_chan("Dev1/ai2", max_val=10, min_val=-10)

    ch_A.ai_term_cfg = nidaqmx.constants.TerminalConfiguration.RSE
    ch_B.ai_term_cfg = nidaqmx.constants.TerminalConfiguration.RSE

    task.timing.cfg_samp_clk_timing(rate=sfreq, sample_mode=constants.AcquisitionType.CONTINUOUS)
    sos = signal.butter(3, 10, 'lp', fs=1000, output='sos')

    while True:
        #if inc%10 == 0:
        #clear_output(wait=True)
        if (inc == 0):
            temp = task.read(number_of_samples_per_channel=2000)
        elif (inc > 0):
            temp = task.read(number_of_samples_per_channel=1000)

        data[0] = data[0] + temp[0]
        data[1] = data[1] + temp[1]

        filtered_A = signal.sosfilt(sos, data[0][(1000 * inc):(1000 * inc) + 2000])
        filtered_B = signal.sosfilt(sos, data[1][(1000 * inc):(1000 * inc) + 2000])

        filtered_A = df(np.transpose(np.array(filtered_A)))
        filtered_B = df(np.transpose(np.array(filtered_B)))

        MA_A = filtered_A.rolling(50).mean()
        MA_B = filtered_B.rolling(50).mean()

        MA_A = list(np.array(MA_A[0].tolist()))
        MA_B = list(np.array(MA_B[0].tolist()))

        for loops in range(49):
            MA_A[loops] = np.mean(MA_A[49:99])
            MA_B[loops] = np.mean(MA_B[49:99])

        for te in range(1000, 2000):
            A_value.append(MA_A[te])
            B_value.append(MA_B[te])

        inc = inc + 1  #중요!!!!

        if count == limit:
            break

        for num in range(1000 * (inc - 1), (1000 * (inc - 1)) + 1000):
            if interval > num:
                continue

            if Motion_occur == 1:
                if flag_1 == 1:
                    Graph_drawing(Frame_start, Frame_end, A_threshold_up, A_threshold_down, B_threshold_up,
                                  B_threshold_down)
                Motion_occur = 0
                count += 1
                print("time :", np.round((time.time() - start), 2), 'sec')
                print('동작 {}회 완료'.format(count))
                if count == limit:
                    playsound("end.mp3")
                    break
                playsound("next.mp3")

                Frame_start = 0
                Frame_end = 0

            if num == 5000:
                print("실험을 시작합니다.")
                playsound("first.mp3")

            if (Frame_start == 0):
                A_mean = np.mean(A_value[num - 799:num - 300])
                B_mean = np.mean(B_value[num - 799:num - 300])

                A_threshold_up = A_mean + float(Initial_threshold)
                A_threshold_down = A_mean - float(Initial_threshold)
                B_threshold_up = B_mean + float(Initial_threshold)
                B_threshold_down = B_mean - float(Initial_threshold)

            if ((A_value[num] > A_threshold_up) or
                    (A_value[num] < A_threshold_down)):

                if (Frame_start == 0):
                    Frame_start = num

                elif (Frame_start != 0):
                    Frame_end = num

            if ((B_value[num] > B_threshold_up) or
                    (B_value[num] < B_threshold_down)):

                if (Frame_start == 0):
                    Frame_start = num

                elif (Frame_start != 0):
                    Frame_end = num

            if Frame_start == 0:
                continue
            if Frame_start == num:
                print("동작 프레임 계산중입니다.")
            #---------------------------------------------------------------------------------
            if (num == Frame_start + 3000):  #센서 A,B 상황 1,2,3,4 전부 걸림
                #-------------------------------------------------------------------------------------------
                diff = Frame_end - Frame_start

                start_point.append(Frame_start)
                end_point.append(Frame_end)

                Motion_occur = 1
                interval = num + int(interval_period * 1000)

    path = '../../../Data/Original Data/' + datetime.today().strftime("%Y%m%d%H%M%S") + '/' + motion + '/' + human + '/'
    if not os.path.isdir(path):
        os.makedirs(path)
    df(A_value).to_csv(path + 'data_A.csv', header=False, index=False)
    df(B_value).to_csv(path + 'data_B.csv', header=False, index=False)

time : 27.71 sec
동작 1회 완료
